<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter13/Detectron2_Chapter13_CustomModelToONNX.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
import torch
print(torch.__version__)

1.13.1+cu116


In [ ]:
!python -m pip -qq install \
'git+https://github.com/facebookresearch/detectron2.git'

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


## Dataset

In [ ]:
!wget -q https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/datasets/braintumors_coco.zip
!unzip -q braintumors_coco.zip

# Get a predictor

In [ ]:
!wget https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/Chapter08/object_detector_hook.zip
!unzip object_detector_hook.zip

--2023-02-19 17:04:27--  https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/Chapter08/object_detector_hook.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/main/Chapter08/object_detector_hook.zip [following]
--2023-02-19 17:04:27--  https://media.githubusercontent.com/media/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/main/Chapter08/object_detector_hook.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 307074938 (293M) [application/zip]
Saving to: ‘object_detector_hook.

In [ ]:
output_dir = "output/object_detector_hook/"

In [ ]:
!unzip {output_dir}model_best.pth.zip

Archive:  output/object_detector_hook/model_best.pth.zip
  inflating: output/object_detector_hook/model_best.pth  


In [ ]:
import detectron2
import pickle
with open(output_dir+"cfg.pickle", "rb") as f:
  cfg = pickle.load(f)

In [ ]:
cfg.MODEL.WEIGHTS = output_dir+"model_best.pth"
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
cfg.MODEL.DEVICE = device

In [ ]:
from detectron2.engine import DefaultPredictor
predictor = DefaultPredictor(cfg)
model = predictor.model

In [ ]:
from detectron2.utils.logger import setup_logger
logger = setup_logger()

In [ ]:
# Some configurations
name_ds = "braintumors_coco"
name_ds_test = name_ds + "_test"
image_root_test = name_ds + "/test"
af = "_annotations.coco.json"
json_file_test = name_ds + "/test/" + af
from detectron2.data.datasets import register_coco_instances

# register test dataset
register_coco_instances(
    name = name_ds_test,
    metadata = {},
    json_file = json_file_test,
    image_root = image_root_test
    )

In [ ]:
from detectron2.data import (
    build_detection_test_loader,
)
tld = build_detection_test_loader(cfg, cfg.DATASETS.TEST[0])
tli = iter(tld)

[02/19 17:04:58 d2.data.datasets.coco]: Loaded 223 images in COCO format from braintumors_coco/test/_annotations.coco.json
[02/19 17:04:58 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|  negative  | 154          |  positive  | 87           |
|            |              |            |              |
|   total    | 241          |            |              |
[02/19 17:04:58 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[02/19 17:04:58 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[02/19 17:04:58 d2.data.common]: Serializing 223 elements to byte tensors and concatenating them all ...
[02/19 17:04:58 d2.data.common]: Serialized dataset takes 0.06 MiB


In [ ]:
def get_model_inputs(tli):
  inputs = next(tli)
  inputs = [{"image": input["image"] for input in inputs}]  
  return inputs

In [ ]:
dummy_inputs = get_model_inputs(tli)
test_inputs = get_model_inputs(tli)

## ONNX

### Export

Note opset_version: https://github.com/facebookresearch/detectron2/blob/main/detectron2/export/__init__.py#L19

In [ ]:
from detectron2.export import TracingAdapter
wrapper = TracingAdapter(model, inputs=dummy_inputs)
wrapper.eval()
model_name = "onnx_model.onnx"

with open(model_name, "wb") as f:
  image = dummy_inputs[0]["image"]
  torch.onnx.export(
      model   = wrapper, 
      args    = (image,), 
      f       = f,
      opset_version = 16
      )

/usr/local/lib/python3.8/dist-packages/detectron2/structures/image_list.py:85: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert t.shape[:-2] == tensors[0].shape[:-2], t.shape
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.8/dist-packages/detectron2/structures/boxes.py:151: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means th

### Locale Error Fix
May need this fix if running on GPU and cannot install onnx

In [ ]:
# fix for locale error if cannot install onnx 
# due to locale error if running GPU runtime
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


### Load the model
Load the exported model and check if it is well formed

In [ ]:
# Check
import onnx
onnx_model = onnx.load(model_name)
onnx.checker.check_model(onnx_model, True)

### Execute the loaded model
- Install onnxruntime
- Create a run-time and execute the onnx loaded model

In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 7.7 MB/s eta 0:00:00


In [ ]:
import onnxruntime as ort
ort_session = ort.InferenceSession(model_name)
input_name = ort_session.get_inputs()[0].name

In [ ]:
outputs = ort_session.run(
    None,
    {input_name: test_inputs[0]["image"].numpy()},
)
print(outputs)

[array([[456.47223, 163.26727, 580.15924, 317.82056],
       [456.27646, 154.84161, 584.78076, 311.30902],
       [492.0383 , 161.29176, 578.70905, 254.3066 ],
       [484.3123 , 161.73778, 575.2621 , 264.60254],
       [451.01773, 200.97081, 546.3406 , 313.3496 ],
       [461.3992 , 195.32011, 546.2074 , 306.30054]], dtype=float32), array([0, 1, 1, 0, 1, 0], dtype=int64), array([0.7608421 , 0.33609974, 0.17438626, 0.10448982, 0.08246013,
       0.08060663], dtype=float32), array([800, 800], dtype=int64)]


In [ ]:
wrapper.outputs_schema(outputs)

[{'instances': Instances(num_instances=6, image_height=800, image_width=800, fields=[pred_boxes: Boxes(tensor([[456.4722, 163.2673, 580.1592, 317.8206],
          [456.2765, 154.8416, 584.7808, 311.3090],
          [492.0383, 161.2918, 578.7090, 254.3066],
          [484.3123, 161.7378, 575.2621, 264.6025],
          [451.0177, 200.9708, 546.3406, 313.3496],
          [461.3992, 195.3201, 546.2074, 306.3005]])), pred_classes: [0 1 1 0 1 0], scores: [0.7608421  0.33609974 0.17438626 0.10448982 0.08246013 0.08060663]])}]

### Validate prediction results
Run prediction using the original model and compare the results

In [ ]:
predictor.model(test_inputs)

[{'instances': Instances(num_instances=6, image_height=800, image_width=800, fields=[pred_boxes: Boxes(tensor([[456.4723, 163.2672, 580.1592, 317.8206],
          [456.2765, 154.8416, 584.7808, 311.3090],
          [492.0383, 161.2917, 578.7090, 254.3066],
          [484.3123, 161.7378, 575.2621, 264.6025],
          [451.0178, 200.9709, 546.3406, 313.3496],
          [461.3992, 195.3202, 546.2075, 306.3005]], device='cuda:0',
         grad_fn=<IndexBackward0>)), scores: tensor([0.7608, 0.3361, 0.1744, 0.1045, 0.0825, 0.0806], device='cuda:0',
         grad_fn=<IndexBackward0>), pred_classes: tensor([0, 1, 1, 0, 1, 0], device='cuda:0')])}]